In [1]:
import pandas as pd
import numpy as np

In [2]:
#pulling in job csv
jobs_ = "../data/Job Postings.csv"
jobs_df = pd.read_csv(jobs_)

In [3]:
#pulling in covid/merchant csv
covidmerchant_ = "../output_files/covidmerchant.csv"
covidmerchant_df = pd.read_csv(covidmerchant_)

In [4]:
#merging jobs/covid/merchants
covidmerchantjobs_df = pd.merge(covidmerchant_df,jobs_df, on=['cityid'], how='outer', suffixes=('_cvm','_jobs'))

In [5]:
#changing cityname to City Name to match cost of living
cv19bizjob_df = covidmerchantjobs_df.rename(columns={"cityname": "City Name"})

In [6]:
#pulling in living csv
living_ = "../output_files/cost_of_living.csv"
living_df = pd.read_csv(living_)

In [7]:
#adding 1 bedroom true cost (if needed)
living_df["1 Bedroom True Cost"]=living_df.iloc[:,[1,3]].sum(axis=1)

#adding 3 bedroom true cost (if needed)
living_df["3 Bedroom True Cost"]=living_df.iloc[:,[2,3]].sum(axis=1)

#adding Average true cost (if needed)
living_df["Avg True Cost"]=living_df.iloc[:,[4,5]].mean(axis=1).round(2)

In [8]:
#merging all info together - covid/merhcnat/jobs/living
#replacing Nan values with "Data Not Available"
allinfo_df = pd.merge(cv19bizjob_df,living_df, on=['City Name'], how='outer')
allinfo_df = allinfo_df.replace({np.NaN: "Data Not Available"})

In [9]:
#keeping cityid lat lon citypop merchantsall(change to%) newcaserate bgpost avetrucost
allinfo_df = allinfo_df.drop(['year',
                        'month',
                        'day',
                        #'month_jobs',
                        #'day_endofweek',
                        'Cost of 1 Bedroom Appartment',
                        'Cost of 3 Bedroom Appartment',
                        'Utilities Per Month',
                        '1 Bedroom True Cost',
                        '3 Bedroom True Cost',
                        'Merchants Difference',
                        'Difference plus 1'], axis = 1)
                        #'year_jobs',
                        #'year_month'],

In [10]:
#pulling in covid/merchant csv
blsgeo_ = "../output_files/BLS_GEO_OCCUPATION_ALL_df_merge.csv"
blsgeo_df = pd.read_csv(blsgeo_,index_col=0)


In [11]:
#keeping geocitystate mwe occtypecode occtype skilllevel howpaid cityid
blsgeo_df = blsgeo_df.drop(['year',
                        'cityname',
                       'stateabbrev',
                        'statefips',
                        'lat',
                        'lon',
                        'city_pop2019',
                        'statename'], axis = 1)

In [12]:
#using a pivot table
ndf = blsgeo_df.pivot(index = "cityid", columns = 'MWE_Occupation Type', values = 'MWE')

In [13]:
#merging last df with pivot df
alldfs = pd.merge(allinfo_df,ndf, on=['cityid'], how='outer')

In [14]:
#replacing all values not accounted for as "Data Not Available"
alldfs = alldfs.replace({'Not Found': "Data Not Available", "-": "Data Not Available"})

In [15]:
#formatting the merchants_all column to percentage
alldfs['merchants_all'] = pd.Series(["{0:.2f}%".format(val * 100) for val in alldfs['merchants_all']], index = alldfs.index)

In [16]:
#import and clean hospital data
health_ = "../output_files/HealthData.csv"
health_df = pd.read_csv(health_)
health_df = health_df.drop(['stateabbrev', 'statename', 'statefips', 'lat', 'lon', 'cityname', 'RANK'], axis = 1)
#merge in hospital data
alldfs = pd.merge(health_df, alldfs, on=['cityid'], how='outer')


In [17]:
#droping more unneeded columns
alldfs = alldfs.drop(['Avg True Cost', 'Total Cost of 3 BHK ', 'Total Cost for 1 BHK ', 'city_pop2019_x', 'temp'], axis = 1)


alldfs.columns




Index(['cityid', 'Health score', 'City Name', 'stateabbrev', 'lat', 'lon',
       'city_pop2019_y', 'merchants_all', 'new_case_rate', 'bg_posts',
       'Average Cost of Living ', 'Architecture and engineering occupations',
       'Arts, design, entertainment, sports, and media occupations',
       'Building and grounds cleaning and maintenance occupations',
       'Business and financial operations occupations',
       'Community and social service occupations',
       'Computer and mathematical occupations',
       'Construction and extraction occupations',
       'Educational instruction and library occupations',
       'Food preparation and serving related occupations',
       'Healthcare practitioners and technical occupations',
       'Healthcare support occupations',
       'Installation, maintenance, and repair occupations',
       'Legal occupations', 'Life, physical, and social science occupations',
       'Management occupations',
       'Office and administrative support oc

In [18]:
# Rename a couple columns
alldfs.rename(columns={'Average Cost of Living ': 'Average Cost of Living', 'Health score': 'Health Score',\
                       'city_pop2019_y':'City pop2019'})


cityid  Health Score         City Name stateabbrev    lat     lon  \
0        1      4.233333       Los Angeles          CA  34.05 -118.24   
1        2      3.784211     New York City          NY  40.71  -74.01   
2        3      4.077778           Chicago          IL  41.88  -87.63   
3        4      4.009091           Houston          TX  29.76  -95.37   
4        5      4.030769           Phoenix          AZ  33.45 -112.07   
5        6      3.953846         San Diego          CA  32.72 -117.16   
6        7      3.375000            Dallas          TX  32.78  -96.80   
7        8      2.600000         Las Vegas          NV  36.17 -115.14   
8        9      3.468750           Seattle          WA  47.61 -122.33   
9       10      4.100000        Fort Worth          TX  32.76  -97.33   
10      11      3.585714       San Antonio          TX  29.42  -98.49   
11      12      4.614286          San Jose          CA  37.34 -121.89   
12      13      3.560000           Detroit          MI  42.33  -83.05   
13      14      3.578571      Philadelphia          PA  39.95  -75.17   
14      15      3.107692          Columbus          OH  39.96  -83.00   
15      16      3.280000            Austin          TX  30.27  -97.74   
16      17      4.000000         Charlotte          NC  35.23  -80.84   
17      18      3.945455      Indianapolis          IN  39.77  -86.16   
18      19      3.744444      Jacksonville          FL  30.33  -81.66   
19      20      3.909091           Memphis          TN  35.15  -90.05   
20      21      4.150000     San Francisco          CA  37.77 -122.42   
21      22      3.400000           El Paso          TX  31.78 -106.44   
22      23      3.705882         Baltimore          MD  39.29  -76.61   
23      24      4.600000          Portland          OR  45.52 -122.68   
24      25      4.233333            Boston          MA  42.36  -71.06   
25      26      3.910000     Oklahoma City          OK  35.47  -97.52   
26      27      3.463636        Louisville          KY  38.25  -85.76   
27      28      3.284615            Denver          CO  39.74 -104.99   
28      29      3.505882        Washington          DC  38.91  -77.04   
29      30      4.200000         Nashville          TN  36.16  -86.78   
30      31      3.830769         Milwaukee          WI  43.04  -87.91   
31      32      3.820000       Albuquerque          NM  35.09 -106.61   
32      33      4.333333            Tucson          AZ  32.22 -110.93   
33      34      3.837500            Fresno          CA  36.75 -119.77   
34      35      3.892308        Sacramento          CA  38.58 -121.49   
35      36      4.178571           Atlanta          GA  33.75  -84.39   
36      37      3.744444       Kansas City          MO  39.10  -94.58   
37      38      4.411111             Miami          FL  25.76  -80.19   
38      39      4.480000           Raleigh          NC  35.78  -78.64   
39      40      4.028571             Omaha          NE  41.25  -96.00   
40      41      3.254545           Oakland          CA  37.80 -122.27   
41      42      3.100000       Minneapolis          MN  44.98  -93.27   
42      43      3.944444             Tampa          FL  27.95  -82.46   
43      44      4.344444       New Orleans          LA  29.95  -90.07   
44      45      3.300000           Wichita          KS  37.69  -97.34   
45      46      4.300000         Cleveland          OH  41.50  -81.69   
46      47      4.200000       Bakersfield          CA  35.37 -119.02   
47      48      4.061538          Honolulu          HI  21.31 -157.86   
48      49      4.408333             Boise          ID  43.62 -116.20   
49      50      3.175000    Salt Lake City          UT  40.76 -111.89   
50      51      5.000000    Virginia Beach          VA  36.85  -75.98   
51      52      4.154545  Colorado Springs          CO  38.83 -104.82   
52      53      3.509091             Tulsa          OK  36.15  -95.99   

    City pop2019 merchants_all  new_case_rate  bg_posts  ... 

In [19]:
# in order to do ranking we need NaN to be in place
alldfs = alldfs.replace({"Data Not Available": float('NaN')})


# now creating another data frame to hold rankings. 
column_list = ['Health score', 'city_pop2019_y', 'merchants_all', 'bg_posts',
       'Average Cost of Living ', 'Architecture and engineering occupations',
       'Arts, design, entertainment, sports, and media occupations',
       'Building and grounds cleaning and maintenance occupations',
       'Business and financial operations occupations',
       'Community and social service occupations',
       'Computer and mathematical occupations',
       'Construction and extraction occupations',
       'Educational instruction and library occupations',
       'Food preparation and serving related occupations',
       'Healthcare practitioners and technical occupations',
       'Healthcare support occupations',
       'Installation, maintenance, and repair occupations',
       'Legal occupations', 'Life, physical, and social science occupations',
       'Management occupations',
       'Office and administrative support occupations',
       'Personal care and service occupations', 'Production occupations',
       'Protective service occupations', 'Sales and related occupations',
       'Transportation and material moving occupations']
for i in column_list:
    print(i)
    alldfs[i+"_RANK"] = alldfs[i].rank(ascending = False, na_option='bottom')
alldfs['new_case_rate'+"_RANK"] = alldfs['new_case_rate'].rank(ascending = True)
alldfs.columns

Health score
city_pop2019_y
merchants_all
bg_posts
Average Cost of Living 
Architecture and engineering occupations
Arts, design, entertainment, sports, and media occupations
Building and grounds cleaning and maintenance occupations
Business and financial operations occupations
Community and social service occupations
Computer and mathematical occupations
Construction and extraction occupations
Educational instruction and library occupations
Food preparation and serving related occupations
Healthcare practitioners and technical occupations
Healthcare support occupations
Installation, maintenance, and repair occupations
Legal occupations
Life, physical, and social science occupations
Management occupations
Office and administrative support occupations
Personal care and service occupations
Production occupations
Protective service occupations
Sales and related occupations
Transportation and material moving occupations


Index(['cityid', 'Health score', 'City Name', 'stateabbrev', 'lat', 'lon',
       'city_pop2019_y', 'merchants_all', 'new_case_rate', 'bg_posts',
       'Average Cost of Living ', 'Architecture and engineering occupations',
       'Arts, design, entertainment, sports, and media occupations',
       'Building and grounds cleaning and maintenance occupations',
       'Business and financial operations occupations',
       'Community and social service occupations',
       'Computer and mathematical occupations',
       'Construction and extraction occupations',
       'Educational instruction and library occupations',
       'Food preparation and serving related occupations',
       'Healthcare practitioners and technical occupations',
       'Healthcare support occupations',
       'Installation, maintenance, and repair occupations',
       'Legal occupations', 'Life, physical, and social science occupations',
       'Management occupations',
       'Office and administrative support oc

In [31]:
# We also wanted to have an overall rank and overall score
rank_columns = [ 'Health score_RANK',
       'city_pop2019_y_RANK', 'merchants_all_RANK', 'bg_posts_RANK',
       'Average Cost of Living _RANK',
       'new_case_rate_RANK']
alldfs['overall_SCORE'] = alldfs[[col for col in rank_columns]].sum(axis=1)
alldfs['overall_RANK'] = alldfs['overall_SCORE'].rank()




City Name  overall_SCORE  overall_RANK
0        Los Angeles          111.5          10.0
1      New York City          113.0          12.0
2            Chicago           92.5           3.0
3            Houston          102.0           4.0
4            Phoenix          157.5          29.0
5          San Diego          144.0          21.0
6             Dallas          135.0          15.5
7          Las Vegas          198.5          39.0
8            Seattle          131.5          13.0
9         Fort Worth          134.5          14.0
10       San Antonio          147.0          22.0
11          San Jose           74.0           1.0
12           Detroit          156.0          25.5
13      Philadelphia          143.5          20.0
14          Columbus          213.5          46.0
15            Austin          109.0           8.0
16         Charlotte          201.0          40.5
17      Indianapolis          155.5          24.0
18      Jacksonville          229.0          49.0
19           Memphis          140.0          18.5
20     San Francisco          110.0           9.0
21           El Paso          265.0          53.0
22         Baltimore          140.0          18.5
23          Portland           91.5           2.0
24            Boston          104.5           6.0
25     Oklahoma City          181.5          34.0
26        Louisville          202.0          42.5
27            Denver          207.5          44.0
28        Washington          136.0          17.0
29         Nashville          112.5          11.0
30         Milwaukee          165.0          31.0
31       Albuquerque          201.0          40.5
32            Tucson          185.5          35.0
33            Fresno          171.0          32.5
34        Sacramento          135.0          15.5
35           Atlanta          156.5          27.0
36       Kansas City          214.0          47.0
37             Miami          154.0          23.0
38           Raleigh          188.0          36.0
39             Omaha          236.0          51.0
40           Oakland          103.0           5.0
41       Minneapolis          191.0          37.0
42             Tampa          157.0          28.0
43       New Orleans          162.5          30.0
44           Wichita          216.0          48.0
45         Cleveland          156.0          25.5
46       Bakersfield          171.0          32.5
47          Honolulu          107.0           7.0
48             Boise          230.0          50.0
49    Salt Lake City          245.5          52.0
50    Virginia Beach          192.0          38.0
51  Colorado Springs          208.0          45.0
52             Tulsa          202.0          42.5

In [32]:
alldfs.to_csv('../output_files/MegaDf.csv')

In [ ]:
#len(x)